In [9]:
# libraries
import itertools
import jedi
import joblib
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pandas_profiling
import pygments
import re
import seaborn as sns
from scipy import stats as st
import sklearn
from tqdm import tqdm
import this

In [2]:
# display for notebooks
%matplotlib inline

In [7]:
# Versions
print("jedi version:", jedi.__version__)
print("numpy version:", np.__version__)
print("pandas version:", pd.__version__)
print("Pygments version:", pygments.__version__)
print("re version:", re.__version__)
print("Seaborn version:", sns.__version__)
print("sklearn version:", sklearn.__version__)

jedi version: 0.16.0
numpy version: 1.18.1
pandas version: 1.0.1
Pygments version: 2.5.2
re version: 2.2.1
Seaborn version: 0.10.0
sklearn version: 0.22.1


In [13]:
# Directories & Files
os.listdir()

# Datasets directory
directory = "./"

In [15]:
datasets

{'license': 'LICENSE',
 'untitled.ipynb': 'Untitled.ipynb',
 'scrapy': 'scrapy',
 'data': 'data.csv',
 'readme.md': 'README.md',
 '.gitignore': '.gitignore',
 'extra': 'extra',
 '.ipynb_checkpoints': '.ipynb_checkpoints',
 '.git': '.git',
 'dgs-reports-archive': 'dgs-reports-archive',
 'notebooks': 'notebooks'}

In [57]:
covid_pt = pd.read_csv("data.csv")

In [58]:
covid_pt.head()
covid_pt.tail()

,data,data_dados,confirmados,confirmados_arsnorte,confirmados_arscentro,confirmados_arslvt,confirmados_arsalentejo,confirmados_arsalgarve,confirmados_acores,confirmados_madeira,...,confirmados_f,confirmados_m,obitos_arsnorte,obitos_arscentro,obitos_arslvt,obitos_arsalentejo,obitos_arsalgarve,obitos_acores,obitos_madeira,obitos_estrangeiro
22,19-03-2020,19-03-2020 00:00,785,381,86,278,2,25,3,1,...,390.0,395.0,0,1,2,0,0,0,0,0
23,20-03-2020,20-03-2020 00:00,1020,506,106,361,2,29,3,1,...,506.0,514.0,1,2,2,0,1,0,0,0
24,21-03-2020,21-03-2020 00:00,1280,644,137,448,3,31,3,5,...,649.0,631.0,4,4,3,0,1,0,0,0
25,22-03-2020,22-03-2020 00:00,1600,825,180,534,5,35,4,7,...,821.0,779.0,5,4,4,0,1,0,0,0
26,23-03-2020,23-03-2020 00:00,2060,1007,238,737,5,42,11,9,...,1080.0,980.0,9,5,8,0,1,0,0,0


In [20]:
covid_pt.dtypes

data                                  object
data_dados                            object
confirmados                            int64
confirmados_arsnorte                   int64
confirmados_arscentro                  int64
confirmados_arslvt                     int64
confirmados_arsalentejo                int64
confirmados_arsalgarve                 int64
confirmados_acores                     int64
confirmados_madeira                    int64
confirmados_estrangeiro              float64
confirmados_novos                      int64
recuperados                            int64
obitos                                 int64
internados                           float64
internados_uci                       float64
lab                                  float64
suspeitos                              int64
vigilancia                           float64
n_confirmados                        float64
cadeias_transmissao                  float64
transmissao_importada                float64
confirmado

In [21]:
covid_pt.shape

(27, 56)

In [23]:
round(covid_pt.describe(), 2)

,confirmados,confirmados_arsnorte,confirmados_arscentro,confirmados_arslvt,confirmados_arsalentejo,confirmados_arsalgarve,confirmados_acores,confirmados_madeira,confirmados_estrangeiro,confirmados_novos,...,confirmados_f,confirmados_m,obitos_arsnorte,obitos_arscentro,obitos_arslvt,obitos_arsalentejo,obitos_arsalgarve,obitos_acores,obitos_madeira,obitos_estrangeiro
count,27.00,27.00,27.00,27.00,27.00,27.00,27.00,27.00,12.00,27.00,...,5.00,5.00,27.00,27.00,27.00,27.0,27.00,27.0,27.0,27.0
mean,333.11,163.33,34.96,119.93,0.70,9.11,1.11,0.89,6.58,76.30,...,689.20,659.80,0.70,0.59,0.78,0.0,0.15,0.0,0.0,0.0
std,552.64,274.29,62.86,194.33,1.49,12.72,2.34,2.29,3.42,119.29,...,271.48,228.55,2.05,1.42,1.78,0.0,0.36,0.0,0.0,0.0
min,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,...,390.00,395.00,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0
25%,5.00,2.50,1.00,1.50,0.00,0.00,0.00,0.00,4.75,2.00,...,506.00,514.00,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0
50%,41.00,27.00,2.00,10.00,0.00,2.00,0.00,0.00,7.50,9.00,...,649.00,631.00,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0
75%,389.50,167.00,41.00,161.00,0.00,13.50,1.00,0.00,9.00,101.50,...,821.00,779.00,0.00,0.00,0.50,0.0,0.00,0.0,0.0,0.0
max,2060.00,1007.00,238.00,737.00,5.00,42.00,11.00,9.00,11.00,460.00,...,1080.00,980.00,9.00,5.00,8.00,0.0,1.00,0.0,0.0,0.0


In [24]:
covid_pt.columns

Index(['data', 'data_dados', 'confirmados', 'confirmados_arsnorte',
       'confirmados_arscentro', 'confirmados_arslvt',
       'confirmados_arsalentejo', 'confirmados_arsalgarve',
       'confirmados_acores', 'confirmados_madeira', 'confirmados_estrangeiro',
       'confirmados_novos', 'recuperados', 'obitos', 'internados',
       'internados_uci', 'lab', 'suspeitos', 'vigilancia', 'n_confirmados',
       'cadeias_transmissao', 'transmissao_importada', 'confirmados_0_9_f',
       'confirmados_0_9_m', 'confirmados_10_19_f', 'confirmados_10_19_m',
       'confirmados_20_29_f', 'confirmados_20_29_m', 'confirmados_30_39_f',
       'confirmados_30_39_m', 'confirmados_40_49_f', 'confirmados_40_49_m',
       'confirmados_50_59_f', 'confirmados_50_59_m', 'confirmados_60_69_f',
       'confirmados_60_69_m', 'confirmados_70_79_f', 'confirmados_70_79_m',
       'confirmados_80_plus_f', 'confirmados_80_plus_m', 'sintomas_tosse',
       'sintomas_febre', 'sintomas_dificuldade_respiratoria',
     

In [37]:
covid19_pt = covid_pt.filter(['data', 'data_dados', 'confirmados', 'confirmados_novos', 
                           'recuperados', 'obitos', 'internados', 'internados_uci', 'lab',
                           'suspeitos', 'vigilancia', 'n_confirmados', 'cadeias_transmissao',
                           'transmissao_importada'])

In [40]:
covid19_pt.shape

(27, 14)

In [41]:
covid19_pt.dtypes

data                      object
data_dados                object
confirmados                int64
confirmados_novos          int64
recuperados                int64
obitos                     int64
internados               float64
internados_uci           float64
lab                      float64
suspeitos                  int64
vigilancia               float64
n_confirmados            float64
cadeias_transmissao      float64
transmissao_importada    float64
dtype: object

In [42]:
covid19_pt.tail()

,data,data_dados,confirmados,confirmados_novos,recuperados,obitos,internados,internados_uci,lab,suspeitos,vigilancia,n_confirmados,cadeias_transmissao,transmissao_importada
22,19-03-2020,19-03-2020 00:00,785,143,3,3,89.0,20.0,488.0,6061,8091.0,4788.0,24.0,71.0
23,20-03-2020,20-03-2020 00:00,1020,235,5,6,126.0,26.0,850.0,7732,9008.0,5862.0,24.0,95.0
24,21-03-2020,21-03-2020 00:00,1280,260,5,12,156.0,35.0,1059.0,9854,13155.0,7515.0,NaN,104.0
25,22-03-2020,22-03-2020 00:00,1600,320,5,14,169.0,41.0,1152.0,11779,12562.0,9027.0,NaN,114.0
26,23-03-2020,23-03-2020 00:00,2060,460,14,23,201.0,47.0,1402.0,13674,11842.0,10212.0,NaN,142.0


In [48]:
covid19_pt[["data", "data_dados"]] = covid19_pt[["data", "data_dados"]].apply(pd.to_datetime)

In [51]:
covid19_pt.dtypes

data                     datetime64[ns]
data_dados               datetime64[ns]
confirmados                       int64
confirmados_novos                 int64
recuperados                       int64
obitos                            int64
internados                      float64
internados_uci                  float64
lab                             float64
suspeitos                         int64
vigilancia                      float64
n_confirmados                   float64
cadeias_transmissao             float64
transmissao_importada           float64
dtype: object

In [52]:
round(covid19_pt.describe(), 2)

,confirmados,confirmados_novos,recuperados,obitos,internados,internados_uci,lab,suspeitos,vigilancia,n_confirmados,cadeias_transmissao,transmissao_importada
count,27.00,27.00,27.00,27.00,19.00,10.00,18.00,27.00,20.00,9.00,15.00,21.00
mean,333.11,76.30,1.63,2.22,95.37,24.30,393.17,2579.11,4934.70,5409.56,12.20,42.29
std,552.64,119.29,3.04,5.49,62.53,12.82,428.84,3927.25,4330.29,2985.08,7.69,42.33
min,0.00,0.00,0.00,0.00,9.00,9.00,30.00,25.00,81.00,1746.00,4.00,2.00
25%,5.00,2.00,0.00,0.00,39.00,17.25,83.00,109.00,483.75,3259.00,6.00,6.00
50%,41.00,9.00,0.00,0.00,89.00,20.00,226.50,375.00,4757.50,4788.00,11.00,33.00
75%,389.50,101.50,3.00,0.50,139.00,32.75,459.50,3469.00,7161.75,7515.00,18.50,62.00
max,2060.00,460.00,14.00,23.00,206.00,47.00,1402.00,13674.00,13155.00,10212.00,24.00,142.00


In [55]:
covid19_pt.drop("data_dados", axis = 1, inplace = True)

In [59]:
covid19_pt.drop("n_confirmados", axis = 1, inplace = True)

In [56]:
covid19_pt["factor_propagacao"] = 

,data,confirmados,confirmados_novos,recuperados,obitos,internados,internados_uci,lab,suspeitos,vigilancia,n_confirmados,cadeias_transmissao,transmissao_importada
0,2020-02-26,0,0,0,0,NaN,NaN,NaN,25,NaN,NaN,NaN,NaN
1,2020-02-27,0,0,0,0,NaN,NaN,NaN,51,NaN,NaN,NaN,NaN
2,2020-02-28,0,0,0,0,NaN,NaN,NaN,59,NaN,NaN,NaN,NaN
3,2020-02-29,0,0,0,0,NaN,NaN,NaN,70,NaN,NaN,NaN,NaN
4,2020-01-03,0,0,0,0,NaN,NaN,NaN,85,NaN,NaN,NaN,NaN
5,2020-02-03,2,2,0,0,NaN,NaN,NaN,85,NaN,NaN,NaN,NaN
6,2020-03-03,4,2,0,0,NaN,NaN,NaN,101,NaN,NaN,NaN,2.0
7,2020-04-03,6,2,0,0,NaN,NaN,NaN,117,81.0,NaN,NaN,4.0
8,2020-05-03,9,3,0,0,9.0,NaN,NaN,147,213.0,NaN,NaN,5.0
9,2020-06-03,13,4,0,0,13.0,NaN,30.0,181,354.0,NaN,5.0,5.0
